# Data Encoder
Uses an autoencoder to reduce the dimensionality of the data.

In [1]:
import tensorflow as tf
print(f"GPU is {'not ' if len(tf.config.list_physical_devices('GPU')) == 0 else ''}available")

2024-04-29 13:12:39.433465: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-29 13:12:40.018157: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


GPU is available


2024-04-29 13:12:40.838192: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 13:12:40.873254: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 13:12:40.873306: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [2]:
import keras

# Data Processing

The data should already be present as `dataset.csv` and `top_unigrams.txt` in the `data` folder.

If they are not present, do the following.
1. Ensure that the VirusTotal reports are present in `data/json` with the format `[LABEL]_[HASH].json`.
3. Run `prepare_data.py`. This will generate the two files needed for this notebook.

In [3]:
import pandas as pd

In [4]:
raw_df = pd.read_csv("../data/dataset.csv")
raw_df

,label,hash,dim-0000,dim-0001,dim-0002,dim-0003,dim-0004,dim-0005,dim-0006,dim-0007,...,dim-9990,dim-9991,dim-9992,dim-9993,dim-9994,dim-9995,dim-9996,dim-9997,dim-9998,dim-9999
0,TRICKBOT,f16631469eb35406ef4049d30c763cadda571b25bbdb45...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,DARKKOMET,d31a7102cbc54447c251ba62760eb484fd0c9fbb8ea54f...,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,INSTALLBRAIN,a5ba68828e571de66675befdf4fbaf26dd226e25c2c703...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,UNKNOWN,c40861e5ebd3c30de810f33c0959aaf5683586fe819998...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,WALDEK,26714e389dbb964ddd764ee4f1bceaf56b18adc8734668...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1268,TRICKBOT,9506421d996290f70689559ee0c09cc074c948fff49547...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1269,NECONYD,0eee965f286f057a3175797590795bbf99fda65dc8d845...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1270,TRICKBOT,0c6aa0ae05d5fa8bf5a8ea95310be73ee60e55a0ce6864...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1271,UNKNOWN,08b4e7389242e3b8c37215a3b972f69193a9a12d5130bf...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


For the training of the model, we don't need the label or the file hash.

In [5]:
df = raw_df.drop(columns=["label", "hash"])
df

,dim-0000,dim-0001,dim-0002,dim-0003,dim-0004,dim-0005,dim-0006,dim-0007,dim-0008,dim-0009,...,dim-9990,dim-9991,dim-9992,dim-9993,dim-9994,dim-9995,dim-9996,dim-9997,dim-9998,dim-9999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1268,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1269,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1270,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1271,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


80% of the dataframe will be saved for training, while 20% will be left for testing.

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(
    df, test_size=0.2, random_state=42
)

In [7]:
X_train.shape

(1018, 10000)

In [8]:
X_test.shape

(255, 10000)

# Model Creation

We will use an autoencoder to reduce the dimensionality of the data.

In [9]:
from keras import layers
from keras.models import Sequential

In [10]:
LAYER_SIZES = [2048, 512, 128, 32]  # The last layer is the center layer

In [11]:
INITIAL_EPOCH = 0

def create_encoder():
    model = Sequential(name="Encoder")
    model.add(keras.Input((df.shape[1],), name="encoder-input"))

    for layer_size in LAYER_SIZES[:-1]:
        model.add(layers.Dense(layer_size, activation="relu"))

    # Add an activity regularizer to make the middle layer sparse
    model.add(layers.Dense(LAYER_SIZES[-1], activation="relu"))
    
    return model


def create_decoder():
    model = Sequential(name="Decoder")
    model.add(keras.Input((LAYER_SIZES[-1],), name="decoder-input"))

    for layer_size in LAYER_SIZES[-2::-1]:  # Starting from second last
        model.add(layers.Dense(layer_size, activation="relu"))
    model.add(layers.Dense(df.shape[1], activation="relu"))
    
    return model


def create_autoencoder(encoder, decoder):
    model = Sequential(name="Autoencoder")
    model.add(keras.Input((df.shape[1],), name="encoder-input"))
    model.add(encoder)
    model.add(decoder)

    model.compile(
        loss="mse",
        optimizer="adam",
        metrics=["mae"]
    )

    return model

In [12]:
encoder = create_encoder()
decoder = create_decoder()
autoencoder = create_autoencoder(encoder, decoder)

2024-04-29 13:12:42.297475: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 13:12:42.297570: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 13:12:42.297617: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 13:12:43.045692: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 13:12:43.045778: I external/local_xla/xla/stream_executor

In [13]:
autoencoder.summary(expand_nested=True)

Model: "Autoencoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Encoder (Sequential)            │ (None, 32)             │    21,600,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense (Dense)              │ (None, 2048)           │    20,482,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_1 (Dense)            │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_2 (Dense)            │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_3 (Dense)            │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Decoder (Sequential)            │ (None, 10000)          │    21,610,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_4 (Dense)            │ (None, 128)            │         4,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_5 (Dense)            │ (None, 512)            │        66,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_6 (Dense)            │ (None, 2048)           │     1,050,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_7 (Dense)            │ (None, 10000)          │    20,490,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 43,211,824 (164.84 MB)

 Trainable params: 43,211,824 (164.84 MB)

 Non-trainable params: 0 (0.00 B)

Define callbacks.

In [14]:
import os

checkpoint_path = "../models/encoder/checkpoint.keras"
checkpoint_dir = os.path.dirname(checkpoint_path)

checkpointer = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    monitor="val_loss",
    save_best_only=True,
    verbose=1
)

In [15]:
early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", patience=10, verbose=1, min_delta=1e-4)

Load latest checkpoint if there is one.

In [16]:
# INITIAL_EPOCH = 54
# autoencoder = keras.models.load_model(checkpoint_path)

Train the model.

In [17]:
NUM_EPOCHS = 200

autoencoder.fit(
    X_train,
    X_train,
    validation_split=0.2,
    initial_epoch=INITIAL_EPOCH,
    epochs=NUM_EPOCHS,
    callbacks=[checkpointer, early_stopping]
)

Epoch 1/200


I0000 00:00:1714367565.043334  916026 service.cc:145] XLA service 0x7f1fc000e180 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1714367565.043418  916026 service.cc:153]   StreamExecutor device (0): Quadro P1000, Compute Capability 6.1
2024-04-29 13:12:45.075737: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-04-29 13:12:45.891269: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


 5/26 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.0192 - mae: 0.0246

I0000 00:00:1714367568.650307  916026 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step - loss: 0.0173 - mae: 0.0254
Epoch 1: val_loss improved from inf to 0.01299, saving model to ../models/encoder/checkpoint.keras
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 229ms/step - loss: 0.0173 - mae: 0.0253 - val_loss: 0.0130 - val_mae: 0.0189
Epoch 2/200
25/26 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0139 - mae: 0.0207
Epoch 2: val_loss improved from 0.01299 to 0.01114, saving model to ../models/encoder/checkpoint.keras
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - loss: 0.0138 - mae: 0.0206 - val_loss: 0.0111 - val_mae: 0.0169
Epoch 3/200
25/26 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0118 - mae: 0.0174
Epoch 3: val_loss improved from 0.01114 to 0.01038, saving model to ../models/encoder/checkpoint.keras
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - loss: 0.0118 - mae: 0.0174 - val_loss: 0.0104 - val_mae: 0.0152
Epoch 4/200
25/26 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0108 - mae: 0.0158
Epoch 4: val_loss improved from 0.01038 to 0.00965, saving mod

Load the best performing model.

In [18]:
autoencoder = keras.models.load_model(checkpoint_path)

Evaluate the autoencoder on the test data.

In [19]:
test_mse, test_mae = autoencoder.evaluate(X_test, X_test, verbose=1)
print(f"Testing MSE: {test_mse:5.5f}")
print(f"Testing MAE: {test_mae:5.5f}")

8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 137ms/step - loss: 0.0074 - mae: 0.0099
Testing MSE: 0.00733
Testing MAE: 0.00988


Get only the encoder part to save.

In [20]:
encoder = autoencoder.get_layer("Encoder")

In [21]:
encoder.summary()

Model: "Encoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 2048)           │    20,482,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 32)             │         4,128 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,600,928 (82.40 MB)

 Trainable params: 21,600,928 (82.40 MB)

 Non-trainable params: 0 (0.00 B)

In [22]:
encoder.save("../models/encoder/encoder.keras")

# Transforming Original Data

In [23]:
transformed_df = encoder.predict(df)

40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step


In [24]:
transformed_df = pd.DataFrame(transformed_df, columns=[f"dim-{i:02d}" for i in range(LAYER_SIZES[-1])])

Add the labels and hashes back to the dataframe.

In [25]:
transformed_df.insert(0, "label", raw_df["label"])
transformed_df.insert(1, "hash", raw_df["hash"])

In [26]:
transformed_df

,label,hash,dim-00,dim-01,dim-02,dim-03,dim-04,dim-05,dim-06,dim-07,...,dim-22,dim-23,dim-24,dim-25,dim-26,dim-27,dim-28,dim-29,dim-30,dim-31
0,TRICKBOT,f16631469eb35406ef4049d30c763cadda571b25bbdb45...,10.802482,0.0,0.0,0.000000,0.000000,12.707260,9.048082,8.721670,...,0.0,13.428146,0.000000,0.0,0.000000,0.000000,0.801215,10.506460,0.024712,0.138089
1,DARKKOMET,d31a7102cbc54447c251ba62760eb484fd0c9fbb8ea54f...,16.677288,0.0,0.0,0.000000,13.798045,0.000000,3.046454,0.000000,...,0.0,11.214973,0.000000,0.0,0.000000,8.833573,6.818852,0.000000,0.000000,0.000000
2,INSTALLBRAIN,a5ba68828e571de66675befdf4fbaf26dd226e25c2c703...,19.391300,0.0,0.0,0.000000,0.000000,7.920878,12.444473,13.511684,...,0.0,6.118662,0.000000,0.0,0.000000,0.000000,4.542971,7.226212,0.000000,1.421442
3,UNKNOWN,c40861e5ebd3c30de810f33c0959aaf5683586fe819998...,1.521830,0.0,0.0,1.158831,0.561619,5.079782,0.000000,4.686564,...,0.0,4.877486,2.747570,0.0,0.000000,0.000000,0.000000,2.483807,2.246307,0.000000
4,WALDEK,26714e389dbb964ddd764ee4f1bceaf56b18adc8734668...,2.786578,0.0,0.0,0.740827,0.000000,3.454260,2.999919,11.366814,...,0.0,8.534671,1.490412,0.0,0.000000,4.255082,2.657059,2.279361,2.798004,3.320712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1268,TRICKBOT,9506421d996290f70689559ee0c09cc074c948fff49547...,15.698259,0.0,0.0,0.000000,13.621684,5.317119,0.000000,0.000000,...,0.0,11.855600,0.000000,0.0,0.000000,0.000000,0.000000,3.220627,0.000000,0.000000
1269,NECONYD,0eee965f286f057a3175797590795bbf99fda65dc8d845...,0.000000,0.0,0.0,1.607330,0.000000,0.000000,4.734159,13.946035,...,0.0,2.253247,4.843779,0.0,0.000000,3.935575,7.204690,11.374994,0.000000,0.000000
1270,TRICKBOT,0c6aa0ae05d5fa8bf5a8ea95310be73ee60e55a0ce6864...,6.559162,0.0,0.0,0.000000,0.000000,0.000000,8.169443,9.100674,...,0.0,10.957123,0.000000,0.0,4.151663,6.178844,7.022230,0.000000,0.000000,0.000000
1271,UNKNOWN,08b4e7389242e3b8c37215a3b972f69193a9a12d5130bf...,0.000000,0.0,0.0,0.000000,1.627770,0.871398,0.000000,7.418483,...,0.0,4.741640,3.873108,0.0,0.000000,0.000000,0.000000,0.000000,4.095950,0.103331


In [27]:
transformed_df.to_csv("../data/encoded-data.csv", index=False)